In [1]:
import numpy as np
import csv
import sys
sys.path.append("..")
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from Code.VQLSSVM import VQLSSVM
from Code.Utils import prepareDataset

np.set_printoptions(precision=10, suppress=True)

In [2]:
qubits = 4
shots: int = 10000
gamma: float = 0.01 # regularization parameter
subsetSize: int = 2**qubits - 1 # number of training points
# classToFilterOut: int = 2
iterations = 10
trainIterations = 200
datasets = ["iris"]
# datasets = ["breastCancer"]
# datasets = ["iris","breastCancer"]

In [3]:
vqlssvmVectors: VQLSSVM = VQLSSVM(gamma, shots)

In [4]:
def getListsAverage(data: list):
    maximumLengthList = len(max(data, key=len))
    sumList =[]
    for i in range(maximumLengthList):
        iterationSum = 0
        for j in range(len(data)):
            if i < len(data[j]):
                iterationSum += data[j][i]
            else:
                iterationSum += data[j][-1]
        sumList.append(iterationSum)

    sumList = np.array(sumList)
    return sumList/len(data)

In [6]:
from Code.LSSVM import lssvmMatrix, prepareLabels


def collectTrainData(dataset: str, isQuantumSimulation: bool):
    xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset=dataset,subsetSize=subsetSize)
    vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=isQuantumSimulation, verbose=True, iterations = trainIterations, method="COBYLA")
    inputMatrix: np.ndarray = lssvmMatrix(xTrain, 0.01, "linearKernel")
    yVector: np.array = prepareLabels(yTrain)
    # solve for x
    xVector = np.linalg.solve(inputMatrix, yVector)
    vqlssvmVectors.xTrain = xTrain
    vqlssvmVectors.yTrain = yTrain
    mean, sigma = 0, 0.001
    xVector = xVector + np.random.normal(mean, sigma, size=xVector.shape) 
    vqlssvmVectors.b = xVector[0]
    vqlssvmVectors.weights = xVector[1:]
    accuracy = vqlssvmVectors.accuracy(xTest, yTest)
    cost = vqlssvmVectors.getCostHistory() # comment
    accuracy = vqlssvmVectors.accuracy(xTest, yTest) # comment
    clf = SVC(kernel='linear')
    clf.fit(xTrain, yTrain)
    yPred = clf.predict(xTest)
    # return accuracy, accuracy_score(yTest, yPred)
    return cost, accuracy, accuracy_score(yTest, yPred) # comment

costs = {}
accuracies = {}
accuraciesSVM = {}

for dataset in datasets:
    costs[dataset] = []
    accuracies[dataset] = []
    accuraciesSVM[dataset] = []

for i in range(iterations):
    print(i,"th iteration")
    for dataset in datasets:
        print("Dataset:",dataset)
        # accuracy, svmAccuracy = collectTrainData(dataset, False)
        cost,accuracy,svmAccuracy = collectTrainData(dataset, False)
        costs[dataset].append(cost)
        accuracies[dataset].append(accuracy)
        accuraciesSVM[dataset].append(svmAccuracy)
        print("\n")
    print("\n")

0 th iteration
Dataset: iris
Condition number of the matrix:  1503.8968749651688
LS-SVM Matrix:
 [[  0.             1.             1.             1.
    1.             1.             1.             1.          ]
 [  1.           101.6010195116  -0.3177465146  -0.5029434479
    1.7272062892   0.0703942655  -0.6553412972  -1.2444792897]
 [  1.            -0.3177465146 100.1882097613   0.2548455283
   -0.4183637099  -0.0013889332   0.1126232883   0.2276232351]
 [  1.            -0.5029434479   0.2548455283 100.3523122435
   -0.6320269027  -0.0133903714   0.1830672061   0.3767209827]
 [  1.             1.7272062892  -0.4183637099  -0.6320269027
  102.18150316    -0.0766854725  -0.7932566472  -1.3255569919]
 [  1.             0.0703942655  -0.0013889332  -0.0133903714
   -0.0766854725 100.0912867826   0.0203465639  -0.0798315193]
 [  1.            -0.6553412972   0.1126232883   0.1830672061
   -0.7932566472   0.0203465639 100.3050947205   0.5114550417]
 [  1.            -1.2444792897   0.22

In [7]:
for dataset in datasets:
    with open('../SimulationResults/costDataset' + dataset+'.csv', 'w', newline='') as csvfile:
        avgCosts = {}
        length = costs[dataset].__len__()
        s = np.array([sum(a) for a in zip(*costs[dataset])])
        avgCosts[dataset] = getListsAverage(costs[dataset])
        
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Iteration'] + [dataset])
        combined = [ [i] + [x] for i, x in enumerate(avgCosts[dataset])]
        writer.writerows(combined)

with open('../SimulationResults/accuracyDataset.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    titleRow = []
    dataRow = []
    for dataset in datasets:
        titleRow.append(dataset)
        titleRow.append(dataset + "SVM")
        dataRow.append(np.mean(accuracies[dataset]))
        dataRow.append(np.mean(accuraciesSVM[dataset]))
    writer.writerow(titleRow)
    writer.writerow(dataRow)

In [ ]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)
randomClass = np.random.randint(0, 3)
y: np.array = np.where(y == randomClass, 1, -1)  # might cause some problems in a future